# ANÁLISIS EXPLORATORIO DE DATOS

## 1. Importar Librerias

In [1]:
# Librerias

import pandas as pd
import os
from pathlib import Path

## 2. Traer Datos

### 2.1. Traer Datos de Defunciones de 1979 - 1991

In [24]:
# Ruta de los archivos
ruta_muertes = Path("data/raw/Muertes")

# Verificar si la carpeta existe
if not ruta_muertes.exists():
    print(f"❌ La carpeta {ruta_muertes} no existe")
else:
    # Listar todos los archivos .txt en la carpeta
    archivos = list(ruta_muertes.glob("Defun*.txt"))
    print(f"📁 Archivos encontrados: {[arch.name for arch in archivos]}")
    
    # Filtrar por años 1979-1991
    archivos_filtrados = [arch for arch in archivos if 1979 <= int(arch.stem[-4:]) <= 1991]
    
    dataframes = {}
    
    for archivo in archivos_filtrados:
        try:
            # Extraer año del nombre del archivo
            año = archivo.stem[-4:]
            
            # Leer archivo (prueba diferentes encodings si es necesario)
            df = pd.read_csv(
                archivo,
                sep='\t',        
                encoding='utf-8', 
                low_memory=False,
                dtype=str
            )
            
            dataframes[f'defun_{año}'] = df
            print(f"✅ {archivo.name}: {len(df)} filas, {len(df.columns)} columnas")
            
        except Exception as e:
            print(f"❌ Error con {archivo.name}: {e}")

📁 Archivos encontrados: ['Defun1979.txt', 'Defun1980.txt', 'Defun1981.txt', 'Defun1982.txt', 'Defun1983.txt', 'Defun1984.txt', 'Defun1985.txt', 'Defun1986.txt', 'Defun1987.txt', 'Defun1988.txt', 'Defun1989.txt', 'Defun1990.txt', 'Defun1991.txt', 'Defun1992.txt', 'Defun1993.txt', 'Defun1994.txt', 'Defun1995.txt', 'Defun1996.txt', 'Defun1997.txt', 'Defun1998.txt', 'Defun1999.txt', 'Defun2000.txt', 'Defun2001.txt', 'Defun2002.txt', 'Defun2003.txt', 'Defun2004.txt', 'Defun2005.txt', 'Defun2006.txt', 'Defun2007.txt', 'Defun2012.txt', 'Defun2013.txt']
✅ Defun1979.txt: 110400 filas, 14 columnas
✅ Defun1980.txt: 125573 filas, 14 columnas
✅ Defun1981.txt: 139505 filas, 14 columnas
✅ Defun1982.txt: 137678 filas, 14 columnas
✅ Defun1983.txt: 140292 filas, 14 columnas
✅ Defun1984.txt: 137189 filas, 14 columnas
✅ Defun1985.txt: 153947 filas, 14 columnas
✅ Defun1986.txt: 146346 filas, 14 columnas
✅ Defun1987.txt: 151957 filas, 14 columnas
✅ Defun1988.txt: 153067 filas, 14 columnas
✅ Defun1989.txt: 1

#### 2.1.1. Consistencia Campos

In [25]:
# Verificar que todas las columnas sean iguales en nombre y orden
nombres_columnas_base = None
columnas_consistentes = True

print("🔍 VERIFICACIÓN DE CONSISTENCIA DE COLUMNAS")
print("=" * 50)

for nombre_df, df in dataframes.items():
    columnas_actuales = df.columns.tolist()
    
    if nombres_columnas_base is None:
        # Establecer la primera como referencia
        nombres_columnas_base = columnas_actuales
        año_base = nombre_df.split('_')[1]
        print(f"📋 DataFrame base (defun_{año_base}): {nombres_columnas_base}")
        print("-" * 50)
    else:
        # Comparar con la base
        if columnas_actuales == nombres_columnas_base:
            print(f"✅ {nombre_df}: Columnas CONSISTENTES")
        else:
            print(f"❌ {nombre_df}: Columnas DIFERENTES")
            print(f"   Esperado: {nombres_columnas_base}")
            print(f"   Encontrado: {columnas_actuales}")
            columnas_consistentes = False

print("=" * 50)
if columnas_consistentes:
    print("🎉 TODAS las columnas son consistentes en nombre y orden")
else:
    print("⚠️  Hay inconsistencias en las columnas")

🔍 VERIFICACIÓN DE CONSISTENCIA DE COLUMNAS
📋 DataFrame base (defun_1979): ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
--------------------------------------------------
❌ defun_1980: Columnas DIFERENTES
   Esperado: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
   Encontrado: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol']
❌ defun_1981: Columnas DIFERENTES
   Esperado: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
   Encontrado: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', '

#### 2.1.2. Estandarización Campos

In [26]:
# Estandarizar todas las columnas a MAYÚSCULAS (como 1979)
print("🔄 ESTANDARIZANDO COLUMNAS A MAYÚSCULAS")
print("=" * 40)

dataframes_estandarizados = {}

for nombre_df, df in dataframes.items():
    # Crear copia del DataFrame
    df_estandar = df.copy()
    
    # Convertir nombres de columnas a MAYÚSCULAS
    df_estandar.columns = [col.upper() for col in df_estandar.columns]
    
    dataframes_estandarizados[nombre_df] = df_estandar
    print(f"✅ {nombre_df}: {list(df.columns)} → {list(df_estandar.columns)}")

# Reemplazar el diccionario original
dataframes = dataframes_estandarizados

🔄 ESTANDARIZANDO COLUMNAS A MAYÚSCULAS
✅ defun_1979: ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
✅ defun_1980: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
✅ defun_1981: ['cod_dpto', 'cod_munic', 'a_defun', 'ano', 'mes', 'sexo', 'gru_ed1', 'est_civil', 'codptore', 'codmunre', 'sit_defun', 'c_bas1', 'cons_exp', 'cau_homol'] → ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EX

In [28]:
# Acceder a defun_1979 y ver primeras 5 filas
defun_1979 = dataframes['defun_1979']

print("👀 PRIMERAS 5 FILAS DE DEFUN_1979")
print("=" * 50)
print(defun_1979.head())

👀 PRIMERAS 5 FILAS DE DEFUN_1979
  COD_DPTO COD_MUNIC A_DEFUN   ANO MES SEXO GRU_ED1 EST_CIVIL CODPTORE  \
0       50       001       1  1979  01    2      14         1       50   
1       50       223       1  1979  01    2      04         1       50   
2       50       223       1  1979  01    2      24         1       50   
3       50       683       1  1979  01    1      18         2       50   
4       50       287       1  1979  01    1      15         1       50   

  CODMUNRE SIT_DEFUN C_BAS1 CONS_EXP CAU_HOMOL  
0      001         1   6370        2       078  
1      223         2   2762        3       043  
2      223         2   2762        3       043  
3      683         2   8259        3       090  
4      287         2   9660        3       101  


#### 2.1.3. Verificar Consistencia 2

In [29]:
# Verificar consistencia después de estandarizar
print("\n🔍 VERIFICACIÓN POST-ESTANDARIZACIÓN")
print("=" * 50)

nombres_columnas_base = None
columnas_consistentes = True

for nombre_df, df in dataframes.items():
    columnas_actuales = df.columns.tolist()
    
    if nombres_columnas_base is None:
        nombres_columnas_base = columnas_actuales
        año_base = nombre_df.split('_')[1]
        print(f"📋 DataFrame base (defun_{año_base}): {nombres_columnas_base}")
        print("-" * 50)
    else:
        if columnas_actuales == nombres_columnas_base:
            print(f"✅ {nombre_df}: Columnas CONSISTENTES")
        else:
            print(f"❌ {nombre_df}: Columnas DIFERENTES")
            columnas_consistentes = False

print("=" * 50)
if columnas_consistentes:
    print("🎉 TODAS las columnas ahora son consistentes")
else:
    print("⚠️  Aún hay inconsistencias")


🔍 VERIFICACIÓN POST-ESTANDARIZACIÓN
📋 DataFrame base (defun_1979): ['COD_DPTO', 'COD_MUNIC', 'A_DEFUN', 'ANO', 'MES', 'SEXO', 'GRU_ED1', 'EST_CIVIL', 'CODPTORE', 'CODMUNRE', 'SIT_DEFUN', 'C_BAS1', 'CONS_EXP', 'CAU_HOMOL']
--------------------------------------------------
✅ defun_1980: Columnas CONSISTENTES
✅ defun_1981: Columnas CONSISTENTES
✅ defun_1982: Columnas CONSISTENTES
✅ defun_1983: Columnas CONSISTENTES
✅ defun_1984: Columnas CONSISTENTES
✅ defun_1985: Columnas CONSISTENTES
✅ defun_1986: Columnas CONSISTENTES
✅ defun_1987: Columnas CONSISTENTES
✅ defun_1988: Columnas CONSISTENTES
✅ defun_1989: Columnas CONSISTENTES
✅ defun_1990: Columnas CONSISTENTES
✅ defun_1991: Columnas CONSISTENTES
🎉 TODAS las columnas ahora son consistentes


#### 2.1.4. Unificar en un sólo DF defunciones 1979-1991

In [30]:
# Unificar todos los DataFrames en uno solo
print("🔄 UNIFICANDO DATAFRAMES (1979-1991)")
print("=" * 50)

# Verificar que todos los DataFrames están listos
print(f"📊 DataFrames a unificar: {len(dataframes)}")
for nombre, df in dataframes.items():
    print(f"   {nombre}: {df.shape}")

# Unificar todos los DataFrames
defun_1979_1991 = pd.concat(dataframes.values(), ignore_index=True)

print(f"\n✅ UNIFICACIÓN COMPLETADA")
print(f"📈 Tamaño del DataFrame unificado: {defun_1979_1991.shape}")
print(f"   - Filas: {len(defun_1979_1991):,}")
print(f"   - Columnas: {len(defun_1979_1991.columns)}")

🔄 UNIFICANDO DATAFRAMES (1979-1991)
📊 DataFrames a unificar: 13
   defun_1979: (110400, 14)
   defun_1980: (125573, 14)
   defun_1981: (139505, 14)
   defun_1982: (137678, 14)
   defun_1983: (140292, 14)
   defun_1984: (137189, 14)
   defun_1985: (153947, 14)
   defun_1986: (146346, 14)
   defun_1987: (151957, 14)
   defun_1988: (153067, 14)
   defun_1989: (154694, 14)
   defun_1990: (154685, 14)
   defun_1991: (163692, 14)

✅ UNIFICACIÓN COMPLETADA
📈 Tamaño del DataFrame unificado: (1869025, 14)
   - Filas: 1,869,025
   - Columnas: 14


In [53]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101


### 2.2. Leer Datos de Codigo Departamento y Codigo Municipio DANE

#### 2.2.1. Conectar archivo de Codigo Departamento y Codigo Municipio DANE

In [51]:
# Ruta del archivo DIVIPOLA de Centros Poblados
ruta_archivo = "data/raw/Referenciales/DIVIPOLA_CentrosPoblados.csv"

# Leer todo como texto para preservar formatos
try:
    divipola = pd.read_csv(
        ruta_archivo, 
        encoding='latin-1',
        sep=';',
        dtype=str  # Esto fuerza todo a texto
    )
    print(f"✅ Archivo leído correctamente (todo como texto)")
    print(f"   - Filas: {len(divipola):,}")
    print(f"   - Columnas: {len(divipola.columns)}")
    
    # Verificar que se mantienen los formatos
    print(f"\n📋 Primeras filas (Código_Departamento):")
    display(divipola.head(10))
    print("\n")
    
    print(f"\n🔍 Tipos de datos:")
    print(divipola.dtypes)
    
except Exception as e:
    print(f"❌ Error: {e}")

✅ Archivo leído correctamente (todo como texto)
   - Filas: 8,421
   - Columnas: 9

📋 Primeras filas (Código_Departamento):


,Código_Departamento,Nombre_Departamento,Código_Municipio,Nombre_Municipio,Código_Entidad,Nombre_Entidad,Tipo,Longitud,Latitud
0,05,ANTIOQUIA,05001,MEDELLÍN,05001000,"MEDELLÍN, DISTRITO ESPECIAL DE CIENCIA, TECNOL...",CM,"-75,581775","6,246631"
1,05,ANTIOQUIA,05001,MEDELLÍN,05001001,PALMITAS,CP,"-75,690573","6,343919"
2,05,ANTIOQUIA,05001,MEDELLÍN,05001004,SANTA ELENA,CP,"-75,501293","6,210599"
3,05,ANTIOQUIA,05001,MEDELLÍN,05001009,ALTAVISTA,CP,"-75,643706","6,221429"
4,05,ANTIOQUIA,05001,MEDELLÍN,05001010,AGUAS FRÍAS,CP,"-75,633948","6,233335"
5,05,ANTIOQUIA,05001,MEDELLÍN,05001013,SAN JOSÉ DEL MANZANILLO,CP,"-75,612136","6,206092"
6,05,ANTIOQUIA,05001,MEDELLÍN,05001014,BARRO BLANCO,CP,"-75,479856","6,236734"
7,05,ANTIOQUIA,05001,MEDELLÍN,05001015,EL CERRO,CP,"-75,489496","6,200314"
8,05,ANTIOQUIA,05001,MEDELLÍN,05001017,EL PATIO,CP,"-75,65111","6,277841"
9,05,ANTIOQUIA,05001,MEDELLÍN,05001018,EL PLACER,CP,"-75,492971","6,224788"





🔍 Tipos de datos:
Código_Departamento    object
Nombre_Departamento    object
Código_Municipio       object
Nombre_Municipio       object
Código_Entidad         object
Nombre_Entidad         object
Tipo                   object
Longitud               object
Latitud                object
dtype: object


### 2.3. Prepara para merge

#### 2.3.1. Corregir códigos 83-18 (Caquetá)

In [54]:
print("\n1. 📝 CORRIGIENDO CÓDIGOS 83 → 18")
print("-" * 30)

defun_1979_1991_corregido = defun_1979_1991.copy()
defun_1979_1991_corregido.loc[defun_1979_1991_corregido['COD_DPTO'] == '83', 'COD_DPTO'] = '18'
print("✅ Códigos 83 → 18 actualizados")


1. 📝 CORRIGIENDO CÓDIGOS 83 → 18
------------------------------
✅ Códigos 83 → 18 actualizados


#### 2.3.2. Hacer merge (join en SQL)

In [55]:
print("\n2. 🔗 HACIENDO MERGE CON CÓDIGOS CORREGIDOS")
print("-" * 40)

defun_1979_1991_con_nombres = defun_1979_1991_corregido.merge(
    divipola_merge,
    on=['COD_DPTO', 'COD_MUNIC'],
    how='left'
)

print(f"✅ Merge completado")
print(f"📊 Dimensiones: {defun_1979_1991_con_nombres.shape}")

# Verificar registros sin match después de la corrección
sin_match = defun_1979_1991_con_nombres['NOMBRE_DEPARTAMENTO'].isna().sum()
print(f"🔍 Registros sin match: {sin_match:,} ({sin_match/len(defun_1979_1991_con_nombres)*100:.2f}%)")


2. 🔗 HACIENDO MERGE CON CÓDIGOS CORREGIDOS
----------------------------------------
✅ Merge completado
📊 Dimensiones: (1869025, 16)
🔍 Registros sin match: 4,271 (0.23%)


#### 2.3.3. Preparar cabeceras municipales

In [56]:
print("\n3. 🏛️ PREPARANDO CABECERAS MUNICIPALES")
print("-" * 35)

cabeceras_municipales = divipola_merge[
    divipola_merge['COD_MUNIC'] == '001'
][['COD_DPTO', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO']].rename(
    columns={
        'NOMBRE_DEPARTAMENTO': 'NOMBRE_DEPTO_CABECERA',
        'NOMBRE_MUNICIPIO': 'NOMBRE_MUNIC_CABECERA'
    }
)

print(f"📋 Cabeceras encontradas: {len(cabeceras_municipales)}")



3. 🏛️ PREPARANDO CABECERAS MUNICIPALES
-----------------------------------
📋 Cabeceras encontradas: 33


#### 2.3.4. Completar registros sin match con cabeceras municipales

In [57]:
print("\n4. 🔄 COMPLETANDO REGISTROS SIN MATCH")
print("-" * 35)

sin_match_mask = defun_1979_1991_con_nombres['NOMBRE_DEPARTAMENTO'].isna()

defun_1979_1991_final = defun_1979_1991_con_nombres.merge(
    cabeceras_municipales,
    on='COD_DPTO',
    how='left'
)

defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_DEPARTAMENTO'] = defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_DEPTO_CABECERA']
defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_MUNICIPIO'] = defun_1979_1991_final.loc[sin_match_mask, 'NOMBRE_MUNIC_CABECERA']

defun_1979_1991_final = defun_1979_1991_final.drop(['NOMBRE_DEPTO_CABECERA', 'NOMBRE_MUNIC_CABECERA'], axis=1)

print("✅ Registros sin match completados")



4. 🔄 COMPLETANDO REGISTROS SIN MATCH
-----------------------------------
✅ Registros sin match completados


#### 2.3.5. Verificación Final

In [58]:
print("\n5. 📊 VERIFICACIÓN FINAL")
print("-" * 25)

sin_match_final = defun_1979_1991_final['NOMBRE_DEPARTAMENTO'].isna().sum()
print(f"🎯 Registros sin match finales: {sin_match_final:,}")

# Mostrar resultado
print(f"\n👀 PRIMERAS FILAS DEL RESULTADO FINAL:")
columnas_mostrar = ['COD_DPTO', 'COD_MUNIC', 'NOMBRE_DEPARTAMENTO', 'NOMBRE_MUNICIPIO', 'ANO']
print(defun_1979_1991_final[columnas_mostrar].head(10))

# Ver códigos que aún no tienen match (si los hay)
if sin_match_final > 0:
    print(f"\n🔍 CÓDIGOS QUE AÚN NO TIENEN MATCH:")
    sin_match_df = defun_1979_1991_final[defun_1979_1991_final['NOMBRE_DEPARTAMENTO'].isna()]
    codigos_sin_match = sin_match_df[['COD_DPTO', 'COD_MUNIC']].drop_duplicates()
    print(codigos_sin_match)
    print(f"Total: {len(codigos_sin_match)} códigos únicos")

# Actualizar el DataFrame principal
defun_1979_1991 = defun_1979_1991_final.copy()
print("\n💾 DataFrame principal actualizado")


5. 📊 VERIFICACIÓN FINAL
-------------------------
🎯 Registros sin match finales: 0

👀 PRIMERAS FILAS DEL RESULTADO FINAL:
  COD_DPTO COD_MUNIC NOMBRE_DEPARTAMENTO   NOMBRE_MUNICIPIO   ANO
0       50       001                META      VILLAVICENCIO  1979
1       50       223                META           CUBARRAL  1979
2       50       223                META           CUBARRAL  1979
3       50       683                META  SAN JUAN DE ARAMA  1979
4       50       287                META      FUENTE DE ORO  1979
5       50       287                META      FUENTE DE ORO  1979
6       50       577                META      PUERTO LLERAS  1979
7       50       577                META      PUERTO LLERAS  1979
8       50       577                META      PUERTO LLERAS  1979
9       50       577                META      PUERTO LLERAS  1979

💾 DataFrame principal actualizado


In [59]:
defun_1979_1991.head()

,COD_DPTO,COD_MUNIC,A_DEFUN,ANO,MES,SEXO,GRU_ED1,EST_CIVIL,CODPTORE,CODMUNRE,SIT_DEFUN,C_BAS1,CONS_EXP,CAU_HOMOL,NOMBRE_DEPARTAMENTO,NOMBRE_MUNICIPIO
0,50,001,1,1979,01,2,14,1,50,001,1,6370,2,078,META,VILLAVICENCIO
1,50,223,1,1979,01,2,04,1,50,223,2,2762,3,043,META,CUBARRAL
2,50,223,1,1979,01,2,24,1,50,223,2,2762,3,043,META,CUBARRAL
3,50,683,1,1979,01,1,18,2,50,683,2,8259,3,090,META,SAN JUAN DE ARAMA
4,50,287,1,1979,01,1,15,1,50,287,2,9660,3,101,META,FUENTE DE ORO
